In [1]:
import pandas as pd
import numpy as np
import cv2
from PIL import Image
from matplotlib import pyplot as plt
import os
import torch
from os import listdir
from os.path import isfile, join
from sklearn import preprocessing
import torchvision.transforms as transforms
import torchvision.models as models
from torch import nn

In [2]:
#Get link to directory
path_to_train_info = os.path.join(os.getenv('DATASETS'), 'ArtRecognision\\train_info.csv')
path_to_train_set = os.path.join(os.getenv('DATASETS'), 'ArtRecognision\\train_2\\')

In [3]:
#Load datasets
train_set = pd.read_csv(path_to_train_info)
train_set

,filename,artist,title,style,genre,date
0,102257.jpg,5b39c876740bfc1cfaf544721c43cac3,Uriel,Color Field Painting,abstract,1955.0
1,75232.jpg,5b39c876740bfc1cfaf544721c43cac3,Vir Heroicus Sublimis,Color Field Painting,abstract,1950.0
2,29855.jpg,96e5bc98488ed589b9bf17ad9fd09371,Night March of a Hundred Demons (left half),Yamato-e,mythological painting,NaN
3,62252.jpg,5b39c876740bfc1cfaf544721c43cac3,"Who’s Afraid of Red, Yellow and Blue II",Color Field Painting,abstract,NaN
4,63861.jpg,5b39c876740bfc1cfaf544721c43cac3,Black Fire I,Color Field Painting,abstract,1963.0
...,...,...,...,...,...,...
79428,23862.jpg,67959e4e5df05b3d9db7c97fd9a0b0f6,Number 547,Abstract Expressionism,NaN,1954.0
79429,25525.jpg,2d72f2000c42051e7c350a39bdce9bc1,Number 13A (Arabesque),Action painting,abstract,1948.0
79430,47038.jpg,f920b951670e3d8dc3c759f12ced7a3e,St. Francis of Assisi,Baroque,religious painting,NaN
79431,9021.jpg,e4183fd3d19c2bca8b7c56d19af92252,Thebe's Revenge,NaN,marina,1982.0


In [4]:
#Get list of images
onlyfiles = [f for f in listdir(path_to_train_set) if isfile(join(path_to_train_set, f))]

In [7]:
#Leave data only for the downloaded images
filtered_train_set = train_set[train_set.filename.isin(onlyfiles)]
final_train_dataset = filtered_train_set[['filename', 'style', 'genre']].dropna()

In [5]:
#image trasformation for resnet
transform = transforms.Compose([transforms.Resize([224, 224]), 
                                transforms.RandomHorizontalFlip(), 
                                transforms.ToTensor(), 
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [43]:
#fuction to crop images with square
def crop_image(image, crop_size):
    width, height = image.size
    cropped_img = image.crop(((width-crop_size)//2, (height-crop_size)//2, (width+crop_size)//2, (height+crop_size)//2))
    return cropped_img

In [44]:
#image pre-processing
crop_size=800

train_image=[]
for name in final_train_dataset['filename']:
    try:
        img=Image.open(path_to_train_set + name)
        c_image = crop_image(img, crop_size)
        t_image = transform(c_image)
        train_image.append(t_image)
    except:
        continue

In [45]:
train_image[0].shape

torch.Size([3, 224, 224])

In [46]:
train_tensor = torch.stack(train_image)

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 4960800768 bytes. Buy new RAM!


In [47]:
discriminator = models.resnet18(pretrained=True)

In [ ]:
from torch import * # импорт всего не одобряется в python, лучше было import nn
discriminator.fc = nn.Sequential(nn.Linear(1000, 512),
                                 nn.Tanh(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 1),
                                 nn.LogSoftmax(dim=1))
# почему LogSoftmax? там же далее используется binary cross entropy
# предлагаю убрать на последнем слое активацию вовсе, а в качестве лосса использовать
# binary_cross_entropy_with_logits

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        # классно оформлено
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        # модель в несколько раз слабее дискриминатора - баланс не сойдётся
        # linear блоки в порождении картинок работают не очень, нужны ConvTranspose2d
        # "свёртки наоборот"
        self.model = nn.Sequential(
            *block(opt.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))), # откуда здесь взялась переменная img_shape?
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape) # откуда здесь взялась переменная img_shape? х2
        return img

In [ ]:
device = torch.device("cuda:0" if use_gpu and torch.cuda.is_available() else "cpu")
generator = Generator().to(device)
discriminator = discriminator.to(device)

In [ ]:
#criterion = nn.CrossEntropyLoss()

In [ ]:
# зачем устанавливать betas, eps и weight decay, которые уже установлены теми же значениями по умолчанию?
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.002, betas=(0.9, 0.999), eps=0.1, weight_decay=0, amsgrad=False)
# здесь передал оптимизатору генератора параметры дискриминатора
g_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.002, betas=(0.9, 0.999), eps=0.1, weight_decay=0, amsgrad=False)

In [ ]:
import torch.utils.data
# train dataloader делают с shuffle = True, чтобы добится большей стохастичности
# чуть больше случайности в выборе сэмплов в батч позволяет чуть проще проскакивать седловые точки
# функции потерь
train_dataloader = torch.utils.data.DataLoader(train_tensor, batch_size=250, shuffle=False)

In [ ]:
num_epochs = 500
generator.train()
discriminator.train()

for epoch in range(num_epochs):
    gen_loss_avg.append(0)
    disc_loss_avg.append(0)
    num_batches = 0
    
    for image_batch, _ in train_dataloader:
        
        # get dataset image and create real and fake labels for use in the loss
        image_batch = image_batch.to(device)
        label_real = torch.ones(image_batch.size(0), device=device)
        label_fake = torch.zeros(image_batch.size(0), device=device)

        # generate a batch of images from samples of the latent prior
        latent = torch.randn(image_batch.size(0), 100, 1, 1, device=device)
        fake_image_batch = generator(latent)
        
        # train discriminator to correctly classify real and fake
        # (detach the computation graph of the generator and the discriminator,
        # so that gradients are not backpropagated into the generator)
        real_pred = discriminator(image_batch).squeeze()
        fake_pred = discriminator(fake_image_batch.detach()).squeeze()
        disc_loss = 0.5 * (
            F.binary_cross_entropy(real_pred, label_real) +
            F.binary_cross_entropy(fake_pred, label_fake))
        
        disc_optimizer.zero_grad()
        disc_loss.backward()
        disc_optimizer.step()
        
        # train generator to output an image that is classified as real
        fake_pred = discriminator(fake_image_batch).squeeze()
        gen_loss = F.binary_cross_entropy(fake_pred, label_real)
        
        gen_optimizer.zero_grad()
        gen_loss.backward()
        gen_optimizer.step()
        
        gen_loss_avg[-1] += gen_loss.item()
        disc_loss_avg[-1] += disc_loss.item()
        num_batches += 1
        
    gen_loss_avg[-1] /= num_batches
    disc_loss_avg[-1] /= num_batches
    print('Epoch [%d / %d] average loss generator vs. discrim.: %f vs. %f' %
          (epoch+1, num_epochs, gen_loss_avg[-1], disc_loss_avg[-1]))